In [1]:
# Import opentrons.execute
import opentrons.execute
# import protocol_api for autocompletion
from opentrons import protocol_api

# for loading custom labware
# import json

from src.pipette_viscous import *

# This is where you establish the API version 
# re-run this will reset the state of the robot (removing all loaded labware)
protocol = opentrons.execute.get_protocol_api('2.11')

In [2]:
protocol.home()

In [2]:
def run(protocol: protocol_api.ProtocolContext):
    "switching lights"
    
    # opentrons-robot-server have to be stopped to obtain control for the lights
    # doing this will block the APP from connecting
    if protocol.rail_lights_on:
        protocol.set_rail_lights(False)
    else:
        protocol.set_rail_lights(True)

In [4]:
# let's try it
run(protocol)

In [5]:
# define labware
incubator = protocol.load_module("temperature module", 11)
rack = incubator.load_labware("opentrons_24_aluminumblock_fisher_1.5ml_centrifuge")    # the rack is mounted upon the temp. module (don't specify slot)
plate = protocol.load_labware("biorad_96_wellplate_200ul_pcr", '3')

# # define custom labware i.e. tipracks
# with open("custom_labware/geb_taller_96_tiprack_10ul.json") as labware_file:
#     labware_def = json.load(labware_file)
#     tip_rack = protocol.load_labware_from_definition(labware_def, '1')

# now, custom labware is automatically included
tip_rack = protocol.load_labware("geb_taller_96_tiprack_10ul", '2')

# pipettes
pipette = protocol.load_instrument("p10_single", "left", tip_racks = [tip_rack])   # here is a p10_multi on the right, but not used here

In [9]:
# do a simple transfer

pipette.transfer(5, rack.wells_by_name()['A1'], plate.wells_by_name()['C1'])

<InstrumentContext: p10_single_v1.5 in LEFT>

In [12]:
transfer_viscous(pipette, protocol, 5, rack.wells_by_name()["A1"], plate.wells_by_name()['A1'])

In [4]:
pipette.pick_up_tip()
pipette.move_to(rack.wells_by_name()["A1"].bottom())

<InstrumentContext: p10_single_v1.5 in LEFT>

In [6]:
pipette.move_to(rack.wells_by_name()['A1'].top(), speed=50)

<InstrumentContext: p10_single_v1.5 in LEFT>

In [27]:
pipette.pick_up_tip()
pipette.aspirate(5, rack.wells_by_name()['A1'].bottom(0.0), rate=0.5)
pipette.dispense(5, plate.wells_by_name()['B5'])
pipette.drop_tip()

<InstrumentContext: p10_single_v1.5 in LEFT>

In [10]:
# transfer_viscous(pipette, protocol, 5, rack.wells_by_name()['A1'], plate.wells_by_name()['D2'])
aspirate_viscous(pipette, protocol, 9, rack.wells_by_name()['A1'], asp_height=1)
dispense_viscous(pipette, protocol, 9, plate.wells_by_name()['E1'])